<h1>ERI1: Guo et al 2013 </h1>
<p>Extract the clinical data from <a href="https://pubmed.ncbi.nlm.nih.gov/37352860/"target="__blank">Guo L, et al. (2023) Null and missense mutations of ERI1 cause a recessive phenotypic dichotomy in humans. Am J Hum Genet.  PMID:37352860</a>.<p>
<p>The authors report a phenotypic dichotomy associated with bi-allelic ERI1 variants by reporting eight affected individuals from seven unrelated families. A severe spondyloepimetaphyseal dysplasia (SEMD) was identified in five affected individuals with missense variants but not in those with bi-allelic null variants, who showed mild intellectual disability and digital anomalies.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
import math
from csv import DictReader
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import re
from pyphetools.creation import *
from pyphetools.visualization import PhenopacketTable, QcVisualizer
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.8.21


In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
PMID="PMID:37352860"
title = "Null and missense mutations of ERI1 cause a recessive phenotypic dichotomy in humans"
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", pmid=PMID, pubmed_title=title)
metadata.default_versions_with_hpo(version=hpo_version)

In [3]:
df = pd.read_excel("input/Guo_2023.xlsx")

In [4]:
patient_id = df.columns

In [5]:
df = df.set_index('Individual').T.reset_index()
df["patient_id"] = df["index"]

In [6]:
scg = SimpleColumnMapperGenerator(df=df,
                                  observed='+',
                                  excluded='-',
                                  hpo_cr=hpo_cr)

In [7]:
column_mapper_d = scg.try_mapping_columns()

In [8]:
from IPython.display import display, HTML
display(HTML(scg.to_html()))

Result,Columns
Mapped,Syndactyly; Cardiac anomaly; Hydronephrosis; Vesicoureteral reflux; Asthma; Conductive hearing impairment; hypernasal speech; Dislocated radial head; Scoliosis; Hip pain; Short stature; Long face; Narrow face; proptosis; Coarse facies; Low-set ears; Limited elbow extension; Finger joint hypermobility; Clinodactyly of the 5th finger; Pes planus; Slender metacarpals; Increased vertebral height; Velopharyngeal insufficiency; Hip dislocation; Patellar dislocation; Narrow forehead; Upslanted palpebral fissure; High palate; Pectus excavatum; Tapered finger; Prominent forehead; Depressed nasal bridge; Micrognathia; Cutaneous syndactyly; Macrotia; Narrow chest; Pulmonary arterial hypertension; Oligodactyly; Tricuspid regurgitation; Platyspondyly; Intrauterine growth retardation; Motor delay; Failure to thrive; Trigonocephaly; Frontal bossing; Sparse hair; Pectus carinatum; Wormian bones; Osteopenia; Delayed skeletal maturation; Inguinal hernia; Ventricular septal defect; Brachycephaly; Anonychia; Strabismus; Low anterior hairline; Epicanthus
Unmapped,index; DNA; Protein; Sex; Age at last follow-up; Weight; Height; Consanguinity; Fetal ultrasound; Gestation age; Birth weight; Birth length; Spine anomaly; Metaphyseal anomaly; Epiphyseal anomaly; Brachydactyly/clinodactyly/camptodactyly; Intellectual disability/developmental delay; Zygomatic hypoplasia; Posteriorly rotated ear; Cupped ear ; patient_id


In [9]:
# Now get the unmapped columns and try option mappers
# The following was only needed to write the notebook
# unmapped_columns = scg.get_unmapped_columns()
# omit_columns = set(column_mapper_d.keys())
# omit_columns.update(['index','DNA','Protein','Age at last follow-up','Consanguinity'])
# auto_results = OptionColumnMapper.autoformat(df=df, concept_recognizer=hpo_cr, omit_columns=omit_columns)
# print(auto_results)

In [10]:
weight_d = {'24\xa0kg (−5 SD)': 'Decreased body weight',
 '26\xa0kg (−5 SD)': 'Decreased body weight',
 '3.3\xa0kg (- 4 SD)': 'Decreased body weight',
 'failure to thrive': 'Failure to thrive'}
excluded_d = {
    '22\xa0kg (8th centile)': 'Decreased body weight',
    '62\xa0kg (85th centile)': 'Decreased body weight',
    '27.6\xa0kg (50th centile)': 'Decreased body weight',
    'normal': 'Failure to thrive',
}
weightMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=weight_d,
                                 excluded_d=excluded_d)
weightMapper.preview_column(df['Weight'])
column_mapper_d['Weight'] = weightMapper

In [11]:
height_d = {'112\xa0cm (−8 SD)': 'Short stature',
 '128\xa0cm (−7 SD)': 'Short stature',
 '50.3\xa0cm (−5 SD)': 'Short stature',
 'short stature': 'Short stature',
 
 '105\xa0cm (<3rd centile)': 'Short stature'}

excluded_d = {
    '130.8\xa0cm (46th centile)': 'Short stature',
 '155\xa0cm (25th centile)': 'Short stature',
 '130\xa0cm (90th centile)': 'Short stature',
}
heightMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=height_d,
                                excluded_d=excluded_d)
heightMapper.preview_column(df['Height'])
column_mapper_d['Height'] = heightMapper

In [12]:
fetal_ultrasound_d = {'hydronephrosis': 'Hydronephrosis',
 'short limbs': 'Limb undergrowth',
 'severe IUGR': 'Intrauterine growth retardation',
 }
excluded = {
    'unremarkable': 'Intrauterine growth retardation'
}
fetal_ultrasoundMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=fetal_ultrasound_d)
fetal_ultrasoundMapper.preview_column(df['Fetal ultrasound'])
column_mapper_d['Fetal ultrasound'] = fetal_ultrasoundMapper

In [13]:
birth_weight_d = {
 '2180\xa0g (−3.2 SD)': 'Small for gestational age',
 '000\xa0g (−3.3 SD)': 'Small for gestational age',}
birth_weightMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=birth_weight_d)
birth_weightMapper.preview_column(df['Birth weight'])
column_mapper_d['Birth weight'] = birth_weightMapper

In [14]:
# Omitting these because we manually curated detailed phenotypes and added them to the input table
#spine_anomaly_d 
#metaphyseal_anomaly_d = {'nan': 'PLACEHOLDER'}
#epiphyseal_anomaly_d = {'+ (wrists)': 'PLACEHOLDER'}

In [15]:
id_gdd_d = {
 'Motor delay': 'Motor delay',
 'Delayed speech and language development': 'Delayed speech and language development',
 'generalized hypotonia': 'Generalized hypotonia',
 'Global developmental delay': 'Global developmental delay',
 'Autism': 'Autism',
 'Intellectual disability mild': 'Intellectual disability, mild',}
id_gddMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=id_gdd_d)
id_gddMapper.preview_column(df['Intellectual disability/developmental delay'])
column_mapper_d['Intellectual disability/developmental delay'] = id_gddMapper

<H2>Variants</H2>

In [16]:
patient_to_variant_d = defaultdict(list)
variant_set = set()   
for _, row in df.iterrows():
    pat_id = str(row['patient_id']) # needed because some IDs come out as ints
    dna_string = row['DNA']    
    fields = dna_string.split(";")
    if len(fields) != 2:
        raise ValueError(f"Malformed variant line {dna_string}")
    for var in fields:
        variant_str = var.strip()
        variant_str = re.sub(r"(c.)?\[","", variant_str)
        variant_str = variant_str.replace("]", "").strip()
        if "8783887" in variant_str:  # this is the structural variant
            variant_set.add("g.8783887_9068578del")
            patient_to_variant_d[pat_id].append("g.8783887_9068578del")
        else:
            variant_str = "c." + variant_str
            variant_set.add(variant_str)
            patient_to_variant_d[pat_id].append(variant_str)
print(f"We got {len(variant_set)} distinct variants.")

We got 12 distinct variants.


In [17]:
validator = VariantValidator(genome_build='hg38', transcript="NM_153332.4")
validated_var_d = defaultdict()

In [18]:
for var in variant_set:
    print(f"Validating {var}")
    if var == 'g.8783887_9068578del':
        sv = StructuralVariant.chromosomal_deletion(cell_contents='Deletion exons 1-4',
                 gene_symbol="ERI1",
                 gene_id="HGNC:23994")
        validated_var_d[var] = sv
    else:
        var_object = validator.encode_hgvs(hgvs=var)
        validated_var_d[var] = var_object
print(f"We got {len(validated_var_d)} variant objects")

Validating c.514C>T
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_153332.4%3Ac.514C>T/NM_153332.4?content-type=application%2Fjson
Validating c.464C>T
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_153332.4%3Ac.464C>T/NM_153332.4?content-type=application%2Fjson
Validating g.8783887_9068578del
Validating c.450A>T
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_153332.4%3Ac.450A>T/NM_153332.4?content-type=application%2Fjson
Validating c.352A>T
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_153332.4%3Ac.352A>T/NM_153332.4?content-type=application%2Fjson
Validating c.401A>G
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_153332.4%3Ac.401A>G/NM_153332.4?content-type=application%2Fjson
Validating c.582+1G>A
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_153332.4%3Ac.582+1G>A/NM_153332.4?content-type=application%2Fjson
Vali

In [19]:
ageMapper = AgeColumnMapper.by_year('Age at last follow-up')
#ageMapper.preview_column(df['Age at last follow-up'])
sexMapper = SexColumnMapper(male_symbol='M', female_symbol='F', column_name='Sex')
#sexMapper.preview_column(df['Sex'])

<h2>Disease diagnosis</h2>
<p>Diseases related to ERI1 are currenttly not represented in OMIM. For this reason, we represent the diagnosis as preliminary below. The authors write:  SEMD was present in the five individuals with at least one missense variant (Table 1). In contrast, three individuals with ERI1 null mutations and the Eri1 KO mice showed a much milder skeletal phenotype without any evidence for SEMD, consistent with the two individuals reported previously, who had homozygous a 284 kb deletion and p.Lys118∗. Notably, of the five individuals with SEMD, three died within 2 years after birth, suggesting missense variants lead to a poor prognosis.</p>

In [20]:
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_mapper_d, 
                        individual_column_name="patient_id", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        pmid=PMID,
                        metadata=metadata)
eri1 = Disease(disease_id='OMIM: 608739', disease_label='ERI1-related disease')
encoder.set_disease(eri1)

In [21]:
individuals = encoder.get_individuals()

In [22]:
for indi in individuals:
    if indi.id not in patient_to_variant_d:
        raise ValueError(f"Error, individual id \"{indi.id}\" without variant data")
    var_list = patient_to_variant_d.get(indi.id)
    if len(var_list) != 2:
        raise ValueError(f"Error, malformed variant list for {indi.id}")
    v1 = var_list[0]
    v2 = var_list[1]
    if v1 == v2:
        if v1 == "g.8783887_9068578del":
            #This is a whole-gene deletion
            eri1_id = "HGNC:23994"
            eri1_sumbol = "ERI1"
            var = StructuralVariant.chromosomal_deletion(cell_contents=v1, gene_id=eri1_id, gene_symbol=eri1_sumbol)
            var.set_homozygous()
            indi.add_variant(var)
        else:
            vvar = validated_var_d.get(v1)
            vvar.set_homozygous()
            indi.add_variant(vvar)
    else:
        vvar1 = validated_var_d.get(v1)
        vvar1.set_heterozygous()
        indi.add_variant(vvar1)
        vvar2 = validated_var_d.get(v2)
        vvar2.set_heterozygous()
        indi.add_variant(vvar2)     

In [23]:
# Validation

In [24]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.BI_ALLELIC)
validated_individuals = cvalidator.get_validated_individual_list()
qc = QcVisualizer(ontology=hpo_ontology)
display(HTML(qc.to_html(validated_individuals)))

ID,Level,Category,Message,HPO Term
PMID_37352860_5,WARNING,REDUNDANT,Abnormal heart morphology is redundant because of Ventricular septal defect,Abnormal heart morphology (HP:0001627)
PMID_37352860_3,WARNING,REDUNDANT,Decreased body weight is redundant because of Small for gestational age,Decreased body weight (HP:0004325)
PMID_37352860_4,WARNING,REDUNDANT,Failure to thrive is listed multiple times,Failure to thrive (HP:0001508)
PMID_37352860_1A,WARNING,REDUNDANT,Hydronephrosis is listed multiple times,Hydronephrosis (HP:0000126)
PMID_37352860_1B,WARNING,REDUNDANT,Hydronephrosis is listed multiple times,Hydronephrosis (HP:0000126)
PMID_37352860_4,WARNING,REDUNDANT,Intrauterine growth retardation is listed multiple times,Intrauterine growth retardation (HP:0001511)
PMID_37352860_4,WARNING,REDUNDANT,Motor delay is listed multiple times,Motor delay (HP:0001270)
PMID_37352860_1A,WARNING,REDUNDANT,Short stature is listed multiple times,Short stature (HP:0004322)
PMID_37352860_3,WARNING,REDUNDANT,Short stature is listed multiple times,Short stature (HP:0004322)
PMID_37352860_4,WARNING,REDUNDANT,Short stature is listed multiple times,Short stature (HP:0004322)


In [ ]:
# Fix the errors

In [25]:
individuals = [vi.get_individual_with_clean_terms() for vi in validated_individuals]
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.BI_ALLELIC)
validated_individuals = cvalidator.get_validated_individual_list()
qc = QcVisualizer(ontology=hpo_ontology)
display(HTML(qc.to_html(validated_individuals)))

In [26]:
from IPython.display import HTML, display

phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
1A (FEMALE; P28Y),ERI1-related disease (OMIM: 608739),NM_153332.4:c.450A>T (heterozygous) NM_153332.4:c.893A>G (heterozygous),Vesicoureteral reflux (HP:0000076); Asthma (HP:0002099); Conductive hearing impairment (HP:0000405); Hypernasal speech (HP:0001611); Dislocated radial head (HP:0003083); Scoliosis (HP:0002650); Hip pain (HP:0030838); Long face (HP:0000276); Narrow face (HP:0000275); Proptosis (HP:0000520); Coarse facial features (HP:0000280); Low-set ears (HP:0000369); Limited elbow extension (HP:0001377); Finger joint hypermobility (HP:0006094); Clinodactyly of the 5th finger (HP:0004209); Pes planus (HP:0001763); Slender metacarpals (HP:0006236); Increased vertebral height (HP:0004570); Cutaneous syndactyly (HP:0012725); Decreased body weight (HP:0004325); Short stature (HP:0004322); Hydronephrosis (HP:0000126); excluded: Platyspondyly (HP:0000926); excluded: Failure to thrive (HP:0001508)
1B (MALE; P26Y),ERI1-related disease (OMIM: 608739),NM_153332.4:c.450A>T (heterozygous) NM_153332.4:c.893A>G (heterozygous),Vesicoureteral reflux (HP:0000076); Asthma (HP:0002099); Conductive hearing impairment (HP:0000405); Hypernasal speech (HP:0001611); Scoliosis (HP:0002650); Coarse facial features (HP:0000280); Low-set ears (HP:0000369); Limited elbow extension (HP:0001377); Pes planus (HP:0001763); Slender metacarpals (HP:0006236); Increased vertebral height (HP:0004570); Velopharyngeal insufficiency (HP:0000220); Hip dislocation (HP:0002827); Patellar dislocation (HP:0002999); Narrow forehead (HP:0000341); Upslanted palpebral fissure (HP:0000582); High palate (HP:0000218); Pectus excavatum (HP:0000767); Tapered finger (HP:0001182); Cutaneous syndactyly (HP:0012725); Decreased body weight (HP:0004325); Short stature (HP:0004322); Hydronephrosis (HP:0000126); excluded: Platyspondyly (HP:0000926); excluded: Failure to thrive (HP:0001508)
2 (MALE; P3Y6M),ERI1-related disease (OMIM: 608739),NM_153332.4:c.464C>T (heterozygous) NM_153332.4:c.893A>C (heterozygous),Abnormal heart morphology (HP:0001627); Short stature (HP:0004322); Prominent forehead (HP:0011220); Depressed nasal bridge (HP:0005280); Micrognathia (HP:0000347); Cutaneous syndactyly (HP:0012725); Macrotia (HP:0000400); Narrow chest (HP:0000774); Pulmonary arterial hypertension (HP:0002092); Limb undergrowth (HP:0009826); Small for gestational age (HP:0001518); excluded: Platyspondyly (HP:0000926); excluded: Failure to thrive (HP:0001508)
3 (MALE; P3Y6M),ERI1-related disease (OMIM: 608739),NM_153332.4:c.401A>G (heterozygous) NM_153332.4:c.895T>C (heterozygous),Syndactyly (HP:0001159); Pulmonary arterial hypertension (HP:0002092); Oligodactyly (HP:0012165); Tricuspid regurgitation (HP:0005180); Platyspondyly (HP:0000926); Short stature (HP:0004322); Small for gestational age (HP:0001518); excluded: Failure to thrive (HP:0001508)
4 (MALE; P2Y),ERI1-related disease (OMIM: 608739),NM_153332.4:c.464C>T (heterozygous) NM_153332.4:c.62C>A (heterozygous),Syndactyly (HP:0001159); Abnormal heart morphology (HP:0001627); Hydronephrosis (HP:0000126); Vesicoureteral reflux (HP:0000076); Low-set ears (HP:0000369); Micrognathia (HP:0000347); Trigonocephaly (HP:0000243); Frontal bossing (HP:0002007); Sparse hair (HP:0008070); Pectus carinatum (HP:0000768); Wormian bones (HP:0002645); Osteopenia (HP:0000938); Delayed skeletal maturation (HP:0002750); Failure to thrive (HP:0001508); Short stature (HP:0004322); Intrauterine growth retardation (HP:0001511); Motor delay (HP:0001270); Delayed speech and language development (HP:0000750); Generalized hypotonia (HP:0001290); Global developmental delay (HP:0001263); excluded: Failure to thrive (HP:0001508); excluded: Short stature (HP:0004322)
5 (FEMALE; P8Y),ERI1-related disease (OMIM: 608739),NM_153332.4:c.514C>T (homozygous),Syndactyly (HP:0001159); Narrow forehead (HP:0000341); Inguinal hernia (HP:0000023); Ventricular septal defect (HP:0001629); Brachycephaly (HP:0000248); Anonychia (HP:000179

In [27]:
output_directory = "phenopackets"
Individual.output_individuals_as_phenopackets(individual_list=individuals, 
                                             metadata=metadata,
                                             outdir=output_directory)

We output 10 GA4GH phenopackets to the directory phenopackets


In [ ]:
# pxf validate --hpo=hp.json *.json
# no errors